In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from ast import literal_eval

A recommender osztály létrehozásakor megkapja a userid-kat, amik alapján létrehozzá a user objektumokat is. mind a recommender, mind a user eltárolja, azokat a filmeket, amiket kedvelt, amik nem tetszettek neki, amiket látott már, és az összes filmet amit feldobott neki korábban. egyelőre ezek még nincsenek pontosan kihasználva, eléggé  egy basic megoldás még csak. lentebb én kézzel hoztam létre a listákat, de futás közben a recommender addLiked metódusát meghívva az adott user-hez hozzá lehet adni a kedvelt filmet. ugyanígy az addDisliked és az addSeen metódusokkal. a reccommend metódus visszaad egy dictionary-t a userId : recommendedMovieId párosokkal, úgy hogy mindenkinek lehetőleg mást ajánljon. a match metódus pedig leellőrzi, hogy van a közös kedvelt film, ha igen visszaadja annak az indexét, ha nem, akkor False.

In [2]:
tags = pd.read_csv('tags.csv')

In [34]:
class Recommender:
    def __init__(self, ids):
        self.n_users = len(ids)
        self.ids = ids
        self.users = {userId:User() for userId in ids}
        self.tags = pd.read_csv('tags.csv')
        self.tags.vector = self.tags.vector.apply(literal_eval)
        self.seen = []
        self.liked = []
        self.disliked = []
        self.all = []
        
    def calcSimilarity(self):
        liked_mean = np.array(self.tags.loc[self.liked]['vector'].tolist()).mean(axis=0)
        similarities = cosine_similarity([liked_mean], np.array(self.tags['vector'].tolist()))[0]
        similarities = pd.DataFrame(similarities, index = tags.index)
        return similarities

    def recommend(self):
        similarities = self.calcSimilarity()
        recommended = {user:self.users[user].recommend(similarities, i+1) for i,user in enumerate(self.users)}
        return recommended
    
    def addLiked(self, movieId, userId):
        self.liked += [movieId]
        self.users[userId].liked += [movieId]
        self.all += [movieId]
        self.users[userId].all += [movieId]
        
    def addDisliked(self, movieId, userId):
        self.disliked += [movieId]
        self.users[userId].disliked += [movieId]
        self.all += [movieId]
        self.users[userId].all += [movieId]
        
    def addSeen(self, movieId, userId):
        self.seen += [movieId]
        self.users[userId].seen += [movieId]
        self.all += [movieId]
        self.users[userId].all += [movieId]
        
    def match(self):
        if self.n_users > 1:
            common = set(self.users[self.ids[0]].liked) & set(self.users[self.ids[1]].liked)
            for i in range(2, self.n_users):
                common = common & set(self.users[self.ids[i]].liked)
            return common.pop() if common else False
        
        return self.users[ids[0]].liked[0] if self.users[ids[0]].liked else False

In [35]:
bool(set([1]))

True

In [36]:
class User:
    def __init__(self):
        self.liked = []
        self.disliked = []
        self.seen = []
        self.all = []
    
    def recommend(self, similarities, n = 1):
        similarities = similarities.drop(index = self.all)
        return similarities.nlargest(n, 0).index.values[-1]

In [47]:
ids = range(3)
rec = Recommender(ids)

In [48]:
tags[tags['title'].str.contains("Basterds")]

,Unnamed: 0.1,Unnamed: 0,title,n_rating,rating_std,rating_mean,vector,years,popularity,year_normed
614,614,614,Inglourious Basterds (2009),23077,0.894771,4.011397,"[0.29287957056381014, -1.1195572870268564, -0....",2009.0,0.776861,0.775281


In [49]:
primer = 864
inception = 654
fellowship = 202
twot = 36
rotk = 227
donnie = 1586
americanbeauty = 295
darkknight = 578
inglorious = 614
pulpfiction = 0
hp1 = 200
hp2 = 210
hp3 = 232

In [58]:
rec.users[0].liked = [primer, inception, donnie]
rec.users[1].liked = [fellowship, twot, inglorious, inception]
rec.users[2].liked = [americanbeauty, pulpfiction, inception]

rec.liked = [primer, inception, donnie, fellowship, twot, inglorious, americanbeauty, pulpfiction]

In [59]:
rec.match()   #az inception mindbe benne van

654

In [53]:
recommendation = rec.recommend()

In [54]:
recommendation

{0: 29, 1: 455, 2: 2230}

In [55]:
tags.loc[list(recommendation.values())]

,Unnamed: 0.1,Unnamed: 0,title,n_rating,rating_std,rating_mean,vector,years,popularity,year_normed
29,29,29,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",34320,0.932862,4.101282,"[-0.3069219306172145, -1.0551139282165616, -0....",2001.0,0.847056,0.595506
455,455,455,Troy (2004),10544,1.008228,3.351574,"[-0.17753686761329548, -1.0060077055930998, -0...",2004.0,0.638328,0.662921
2230,2230,2234,Dogma (1999),17152,1.010336,3.625321,"[-0.3967678992023353, -1.0884400020358314, -0....",1999.0,0.724382,0.550562
